Walk Length 10

In [6]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Function to load edge list from a file
def load_edgelist(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            edge = line.strip().split()
            edges.append((edge[0], edge[1]))
    return edges

# Function to sample a subgraph using BFS
def sample_subgraph_bfs(graph, start_node=None, number_of_nodes=1000, seed=None):
    if start_node is None:
        nodes = list(graph.nodes())
        random.Random(seed).shuffle(nodes)
        start_node = nodes[0]

    sampled_nodes = set([start_node])
    node_queue = [start_node]

    while len(sampled_nodes) < number_of_nodes and node_queue:
        current = node_queue.pop(0)
        neighbors = sorted(list(graph.neighbors(current)))
        random.Random(seed).shuffle(neighbors)

        for neighbor in neighbors:
            if len(sampled_nodes) >= number_of_nodes:
                break
            if neighbor not in sampled_nodes:
                sampled_nodes.add(neighbor)
                node_queue.append(neighbor)

    return graph.subgraph(sampled_nodes)

# Function to generate edge embeddings
def generate_edge_embeddings(edges, embedding_function):
    edge_embeddings = []
    for u, v in edges:
        u_embedding = embedding_function(u)
        v_embedding = embedding_function(v)
        edge_embedding = np.multiply(u_embedding, v_embedding)
        edge_embeddings.append(edge_embedding)
    return np.array(edge_embeddings)

# Function to split the graph into train and test sets, ensuring connectivity
def split_graph_with_connectivity(graph, fraction_to_remove=0.1, seed=None):
    edge_list = list(graph.edges())
    num_edges_to_remove = int(fraction_to_remove * graph.number_of_edges())

    while True:
        random.Random(seed).shuffle(edge_list)
        edges_to_remove = edge_list[:num_edges_to_remove]
        G_train = graph.copy()
        G_train.remove_edges_from(edges_to_remove)
        if nx.is_connected(G_train):
            break

    G_test = graph.copy()
    G_test.remove_edges_from(G_train.edges())

    return G_train, G_test, edges_to_remove

# Function to generate positive and negative samples
def generate_samples(graph, seed=None):
    positive_samples = list(graph.edges())
    negative_samples = []
    all_nodes = sorted(list(graph.nodes()))

    while len(negative_samples) < len(positive_samples):
        node_pair = random.sample(all_nodes, 2)
        if not graph.has_edge(*node_pair):
            negative_samples.append(tuple(sorted(node_pair)))

    return positive_samples, negative_samples

# Function to generate Node2Vec embeddings
def generate_node2vec_embeddings(graph, dimensions=128, num_walks=30, walk_length=10, p=1.0, q=1.0, seed=None):
    # Function to perform random walk based on Node2Vec algorithm
    def node2vec_walk(node):
        walk = [node]
        for _ in range(walk_length):
            neighbors = list(graph.neighbors(walk[-1]))
            if len(neighbors) > 0:
                if len(walk) == 1:
                    walk.append(random.choice(neighbors))
                else:
                    prev_node = walk[-2]
                    probs = []
                    for neighbor in neighbors:
                        if neighbor == prev_node:
                            probs.append(1 / p)
                        elif graph.has_edge(prev_node, neighbor):
                            probs.append(1)
                        else:
                            probs.append(1 / q)
                    probs = np.array(probs)
                    probs /= np.sum(probs)
                    walk.append(np.random.choice(neighbors, p=probs))
            else:
                break
        return walk

    # Generating random walks
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.seed(seed)
        random.shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(node))

    # Learning embeddings using Word2Vec with a fixed seed
    embeddings = {}
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=1, sg=1, workers=1, seed=seed)
    for node in nodes:
        embeddings[node] = model.wv[node]

    # Function to get node embeddings
    def embedding_function(u):
        return embeddings[u]

    return model, embedding_function

# Grid search over p, q ∈ {0.25, 0.50, 1, 2}
p_values = [0.25, 0.5, 1, 2,3,4]
q_values = [0.25, 0.5, 1, 2,3,4]

# DataFrame to store the results
results_df = pd.DataFrame(columns=['seed', 'p', 'q', 'roc_auc'])

for seed in range(1):
    # Set the new random seed for all functions
    np.random.seed(seed)
    random.seed(seed)

    # Load the data and create the graph
    edge_list = load_edgelist('facebook_combined.txt')  # Replace with your actual file path
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # Sample subgraph, split into train/test, generate samples
    subgraph = sample_subgraph_bfs(G, number_of_nodes=300, seed=seed)
    G_train, G_test, _ = split_graph_with_connectivity(subgraph, seed=seed)
    positive_samples_train, negative_samples_train = generate_samples(G_train, seed=seed)
    positive_samples_test, negative_samples_test = generate_samples(G_test, seed=seed)

    for p in p_values:
        for q in q_values:
            # Node2Vec embeddings
            _, embedding_function = generate_node2vec_embeddings(G_train, p=p, q=q, seed=seed)

            # Generate edge embeddings
            edge_embeddings_train = generate_edge_embeddings(positive_samples_train + negative_samples_train, embedding_function)
            edge_embeddings_test = generate_edge_embeddings(positive_samples_test + negative_samples_test, embedding_function)

            # Logistic Regression Model
            labels_train = np.array([1] * len(positive_samples_train) + [0] * len(negative_samples_train))
            lr_clf = LogisticRegressionCV(cv=10, max_iter=2000, scoring="roc_auc", random_state=seed)
            classifier = lr_clf.fit(edge_embeddings_train, labels_train)

            # Evaluate the model
            labels_test = np.array([1] * len(positive_samples_test) + [0] * len(negative_samples_test))
            test_predictions = classifier.predict_proba(edge_embeddings_test)[:, 1]
            test_roc_auc = roc_auc_score(labels_test, test_predictions)

            # Store results
            results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)

# Print DataFrame
print(results_df)

# Find the best combination of p and q based on average ROC AUC
best_combination = results_df.groupby(['p', 'q'])['roc_auc'].mean().idxmax()
print(f"Best combination of p and q: {best_combination}")


C:\Users\ayush\AppData\Local\Temp\ipykernel_45780\677807264.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)


   seed     p     q   roc_auc
0     0  0.25  0.25  0.887377
1     0  0.25  0.50  0.909079
2     0  0.25  1.00  0.893868
3     0  0.25  2.00  0.903424
4     0  0.25  3.00  0.884476
5     0  0.25  4.00  0.896868
6     0  0.50  0.25  0.907702
7     0  0.50  0.50  0.913455
8     0  0.50  1.00  0.903227
9     0  0.50  2.00  0.890295
10    0  0.50  3.00  0.915996
11    0  0.50  4.00  0.909817
12    0  1.00  0.25  0.898982
13    0  1.00  0.50  0.909308
14    0  1.00  1.00  0.903129
15    0  1.00  2.00  0.921651
16    0  1.00  3.00  0.909554
17    0  1.00  4.00  0.900310
18    0  2.00  0.25  0.898441
19    0  2.00  0.50  0.913914
20    0  2.00  1.00  0.895720
21    0  2.00  2.00  0.894327
22    0  2.00  3.00  0.887689
23    0  2.00  4.00  0.901392
24    0  3.00  0.25  0.912242
25    0  3.00  0.50  0.924241
26    0  3.00  1.00  0.909849
27    0  3.00  2.00  0.911013
28    0  3.00  3.00  0.908161
29    0  3.00  4.00  0.898179
30    0  4.00  0.25  0.910980
31    0  4.00  0.50  0.914734
32    0  4

Walk Length 5

In [7]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Function to load edge list from a file
def load_edgelist(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            edge = line.strip().split()
            edges.append((edge[0], edge[1]))
    return edges

# Function to sample a subgraph using BFS
def sample_subgraph_bfs(graph, start_node=None, number_of_nodes=1000, seed=None):
    if start_node is None:
        nodes = list(graph.nodes())
        random.Random(seed).shuffle(nodes)
        start_node = nodes[0]

    sampled_nodes = set([start_node])
    node_queue = [start_node]

    while len(sampled_nodes) < number_of_nodes and node_queue:
        current = node_queue.pop(0)
        neighbors = sorted(list(graph.neighbors(current)))
        random.Random(seed).shuffle(neighbors)

        for neighbor in neighbors:
            if len(sampled_nodes) >= number_of_nodes:
                break
            if neighbor not in sampled_nodes:
                sampled_nodes.add(neighbor)
                node_queue.append(neighbor)

    return graph.subgraph(sampled_nodes)

# Function to generate edge embeddings
def generate_edge_embeddings(edges, embedding_function):
    edge_embeddings = []
    for u, v in edges:
        u_embedding = embedding_function(u)
        v_embedding = embedding_function(v)
        edge_embedding = np.multiply(u_embedding, v_embedding)
        edge_embeddings.append(edge_embedding)
    return np.array(edge_embeddings)

# Function to split the graph into train and test sets, ensuring connectivity
def split_graph_with_connectivity(graph, fraction_to_remove=0.1, seed=None):
    edge_list = list(graph.edges())
    num_edges_to_remove = int(fraction_to_remove * graph.number_of_edges())

    while True:
        random.Random(seed).shuffle(edge_list)
        edges_to_remove = edge_list[:num_edges_to_remove]
        G_train = graph.copy()
        G_train.remove_edges_from(edges_to_remove)
        if nx.is_connected(G_train):
            break

    G_test = graph.copy()
    G_test.remove_edges_from(G_train.edges())

    return G_train, G_test, edges_to_remove

# Function to generate positive and negative samples
def generate_samples(graph, seed=None):
    positive_samples = list(graph.edges())
    negative_samples = []
    all_nodes = sorted(list(graph.nodes()))

    while len(negative_samples) < len(positive_samples):
        node_pair = random.sample(all_nodes, 2)
        if not graph.has_edge(*node_pair):
            negative_samples.append(tuple(sorted(node_pair)))

    return positive_samples, negative_samples

# Function to generate Node2Vec embeddings
def generate_node2vec_embeddings(graph, dimensions=128, num_walks=30, walk_length=5, p=1.0, q=1.0, seed=None):
    # Function to perform random walk based on Node2Vec algorithm
    def node2vec_walk(node):
        walk = [node]
        for _ in range(walk_length):
            neighbors = list(graph.neighbors(walk[-1]))
            if len(neighbors) > 0:
                if len(walk) == 1:
                    walk.append(random.choice(neighbors))
                else:
                    prev_node = walk[-2]
                    probs = []
                    for neighbor in neighbors:
                        if neighbor == prev_node:
                            probs.append(1 / p)
                        elif graph.has_edge(prev_node, neighbor):
                            probs.append(1)
                        else:
                            probs.append(1 / q)
                    probs = np.array(probs)
                    probs /= np.sum(probs)
                    walk.append(np.random.choice(neighbors, p=probs))
            else:
                break
        return walk

    # Generating random walks
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.seed(seed)
        random.shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(node))

    # Learning embeddings using Word2Vec with a fixed seed
    embeddings = {}
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=1, sg=1, workers=1, seed=seed)
    for node in nodes:
        embeddings[node] = model.wv[node]

    # Function to get node embeddings
    def embedding_function(u):
        return embeddings[u]

    return model, embedding_function

# Grid search over p, q ∈ {0.25, 0.50, 1, 2}
p_values = [0.25, 0.5, 1, 2,3,4]
q_values = [0.25, 0.5, 1, 2,3,4]

# DataFrame to store the results
results_df = pd.DataFrame(columns=['seed', 'p', 'q', 'roc_auc'])

for seed in range(1):
    # Set the new random seed for all functions
    np.random.seed(seed)
    random.seed(seed)

    # Load the data and create the graph
    edge_list = load_edgelist('facebook_combined.txt')  # Replace with your actual file path
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # Sample subgraph, split into train/test, generate samples
    subgraph = sample_subgraph_bfs(G, number_of_nodes=300, seed=seed)
    G_train, G_test, _ = split_graph_with_connectivity(subgraph, seed=seed)
    positive_samples_train, negative_samples_train = generate_samples(G_train, seed=seed)
    positive_samples_test, negative_samples_test = generate_samples(G_test, seed=seed)

    for p in p_values:
        for q in q_values:
            # Node2Vec embeddings
            _, embedding_function = generate_node2vec_embeddings(G_train, p=p, q=q, seed=seed)

            # Generate edge embeddings
            edge_embeddings_train = generate_edge_embeddings(positive_samples_train + negative_samples_train, embedding_function)
            edge_embeddings_test = generate_edge_embeddings(positive_samples_test + negative_samples_test, embedding_function)

            # Logistic Regression Model
            labels_train = np.array([1] * len(positive_samples_train) + [0] * len(negative_samples_train))
            lr_clf = LogisticRegressionCV(cv=10, max_iter=2000, scoring="roc_auc", random_state=seed)
            classifier = lr_clf.fit(edge_embeddings_train, labels_train)

            # Evaluate the model
            labels_test = np.array([1] * len(positive_samples_test) + [0] * len(negative_samples_test))
            test_predictions = classifier.predict_proba(edge_embeddings_test)[:, 1]
            test_roc_auc = roc_auc_score(labels_test, test_predictions)

            # Store results
            results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)

# Print DataFrame
print(results_df)

# Find the best combination of p and q based on average ROC AUC
best_combination = results_df.groupby(['p', 'q'])['roc_auc'].mean().idxmax()
print(f"Best combination of p and q: {best_combination}")


C:\Users\ayush\AppData\Local\Temp\ipykernel_45780\1798957508.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)


   seed     p     q   roc_auc
0     0  0.25  0.25  0.906489
1     0  0.25  0.50  0.910734
2     0  0.25  1.00  0.891623
3     0  0.25  2.00  0.874707
4     0  0.25  3.00  0.904490
5     0  0.25  4.00  0.892754
6     0  0.50  0.25  0.906817
7     0  0.50  0.50  0.906014
8     0  0.50  1.00  0.891147
9     0  0.50  2.00  0.903031
10    0  0.50  3.00  0.901277
11    0  0.50  4.00  0.881591
12    0  1.00  0.25  0.916717
13    0  1.00  0.50  0.910587
14    0  1.00  1.00  0.905276
15    0  1.00  2.00  0.899195
16    0  1.00  3.00  0.900752
17    0  1.00  4.00  0.907817
18    0  2.00  0.25  0.910308
19    0  2.00  0.50  0.906096
20    0  2.00  1.00  0.914685
21    0  2.00  2.00  0.900113
22    0  2.00  3.00  0.915291
23    0  2.00  4.00  0.906194
24    0  3.00  0.25  0.911128
25    0  3.00  0.50  0.910866
26    0  3.00  1.00  0.910767
27    0  3.00  2.00  0.908194
28    0  3.00  3.00  0.913898
29    0  3.00  4.00  0.914718
30    0  4.00  0.25  0.908931
31    0  4.00  0.50  0.914209
32    0  4

Walk Length 15

In [8]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Function to load edge list from a file
def load_edgelist(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            edge = line.strip().split()
            edges.append((edge[0], edge[1]))
    return edges

# Function to sample a subgraph using BFS
def sample_subgraph_bfs(graph, start_node=None, number_of_nodes=1000, seed=None):
    if start_node is None:
        nodes = list(graph.nodes())
        random.Random(seed).shuffle(nodes)
        start_node = nodes[0]

    sampled_nodes = set([start_node])
    node_queue = [start_node]

    while len(sampled_nodes) < number_of_nodes and node_queue:
        current = node_queue.pop(0)
        neighbors = sorted(list(graph.neighbors(current)))
        random.Random(seed).shuffle(neighbors)

        for neighbor in neighbors:
            if len(sampled_nodes) >= number_of_nodes:
                break
            if neighbor not in sampled_nodes:
                sampled_nodes.add(neighbor)
                node_queue.append(neighbor)

    return graph.subgraph(sampled_nodes)

# Function to generate edge embeddings
def generate_edge_embeddings(edges, embedding_function):
    edge_embeddings = []
    for u, v in edges:
        u_embedding = embedding_function(u)
        v_embedding = embedding_function(v)
        edge_embedding = np.multiply(u_embedding, v_embedding)
        edge_embeddings.append(edge_embedding)
    return np.array(edge_embeddings)

# Function to split the graph into train and test sets, ensuring connectivity
def split_graph_with_connectivity(graph, fraction_to_remove=0.1, seed=None):
    edge_list = list(graph.edges())
    num_edges_to_remove = int(fraction_to_remove * graph.number_of_edges())

    while True:
        random.Random(seed).shuffle(edge_list)
        edges_to_remove = edge_list[:num_edges_to_remove]
        G_train = graph.copy()
        G_train.remove_edges_from(edges_to_remove)
        if nx.is_connected(G_train):
            break

    G_test = graph.copy()
    G_test.remove_edges_from(G_train.edges())

    return G_train, G_test, edges_to_remove

# Function to generate positive and negative samples
def generate_samples(graph, seed=None):
    positive_samples = list(graph.edges())
    negative_samples = []
    all_nodes = sorted(list(graph.nodes()))

    while len(negative_samples) < len(positive_samples):
        node_pair = random.sample(all_nodes, 2)
        if not graph.has_edge(*node_pair):
            negative_samples.append(tuple(sorted(node_pair)))

    return positive_samples, negative_samples

# Function to generate Node2Vec embeddings
def generate_node2vec_embeddings(graph, dimensions=128, num_walks=30, walk_length=15, p=1.0, q=1.0, seed=None):
    # Function to perform random walk based on Node2Vec algorithm
    def node2vec_walk(node):
        walk = [node]
        for _ in range(walk_length):
            neighbors = list(graph.neighbors(walk[-1]))
            if len(neighbors) > 0:
                if len(walk) == 1:
                    walk.append(random.choice(neighbors))
                else:
                    prev_node = walk[-2]
                    probs = []
                    for neighbor in neighbors:
                        if neighbor == prev_node:
                            probs.append(1 / p)
                        elif graph.has_edge(prev_node, neighbor):
                            probs.append(1)
                        else:
                            probs.append(1 / q)
                    probs = np.array(probs)
                    probs /= np.sum(probs)
                    walk.append(np.random.choice(neighbors, p=probs))
            else:
                break
        return walk

    # Generating random walks
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.seed(seed)
        random.shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(node))

    # Learning embeddings using Word2Vec with a fixed seed
    embeddings = {}
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=1, sg=1, workers=1, seed=seed)
    for node in nodes:
        embeddings[node] = model.wv[node]

    # Function to get node embeddings
    def embedding_function(u):
        return embeddings[u]

    return model, embedding_function

# Grid search over p, q ∈ {0.25, 0.50, 1, 2}
p_values = [0.25, 0.5, 1, 2,3,4]
q_values = [0.25, 0.5, 1, 2,3,4]

# DataFrame to store the results
results_df = pd.DataFrame(columns=['seed', 'p', 'q', 'roc_auc'])

for seed in range(1):
    # Set the new random seed for all functions
    np.random.seed(seed)
    random.seed(seed)

    # Load the data and create the graph
    edge_list = load_edgelist('facebook_combined.txt')  # Replace with your actual file path
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # Sample subgraph, split into train/test, generate samples
    subgraph = sample_subgraph_bfs(G, number_of_nodes=300, seed=seed)
    G_train, G_test, _ = split_graph_with_connectivity(subgraph, seed=seed)
    positive_samples_train, negative_samples_train = generate_samples(G_train, seed=seed)
    positive_samples_test, negative_samples_test = generate_samples(G_test, seed=seed)

    for p in p_values:
        for q in q_values:
            # Node2Vec embeddings
            _, embedding_function = generate_node2vec_embeddings(G_train, p=p, q=q, seed=seed)

            # Generate edge embeddings
            edge_embeddings_train = generate_edge_embeddings(positive_samples_train + negative_samples_train, embedding_function)
            edge_embeddings_test = generate_edge_embeddings(positive_samples_test + negative_samples_test, embedding_function)

            # Logistic Regression Model
            labels_train = np.array([1] * len(positive_samples_train) + [0] * len(negative_samples_train))
            lr_clf = LogisticRegressionCV(cv=10, max_iter=2000, scoring="roc_auc", random_state=seed)
            classifier = lr_clf.fit(edge_embeddings_train, labels_train)

            # Evaluate the model
            labels_test = np.array([1] * len(positive_samples_test) + [0] * len(negative_samples_test))
            test_predictions = classifier.predict_proba(edge_embeddings_test)[:, 1]
            test_roc_auc = roc_auc_score(labels_test, test_predictions)

            # Store results
            results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)

# Print DataFrame
print(results_df)

# Find the best combination of p and q based on average ROC AUC
best_combination = results_df.groupby(['p', 'q'])['roc_auc'].mean().idxmax()
print(f"Best combination of p and q: {best_combination}")


C:\Users\ayush\AppData\Local\Temp\ipykernel_45780\3199962613.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)


   seed     p     q   roc_auc
0     0  0.25  0.25  0.907456
1     0  0.25  0.50  0.888967
2     0  0.25  1.00  0.880395
3     0  0.25  2.00  0.883132
4     0  0.25  3.00  0.867659
5     0  0.25  4.00  0.877657
6     0  0.50  0.25  0.899261
7     0  0.50  0.50  0.905293
8     0  0.50  1.00  0.897556
9     0  0.50  2.00  0.892754
10    0  0.50  3.00  0.894343
11    0  0.50  4.00  0.874281
12    0  1.00  0.25  0.900687
13    0  1.00  0.50  0.903654
14    0  1.00  1.00  0.905194
15    0  1.00  2.00  0.898261
16    0  1.00  3.00  0.891426
17    0  1.00  4.00  0.893852
18    0  2.00  0.25  0.891541
19    0  2.00  0.50  0.905817
20    0  2.00  1.00  0.903719
21    0  2.00  2.00  0.892147
22    0  2.00  3.00  0.918864
23    0  2.00  4.00  0.890688
24    0  3.00  0.25  0.902277
25    0  3.00  0.50  0.899261
26    0  3.00  1.00  0.910046
27    0  3.00  2.00  0.907292
28    0  3.00  3.00  0.906653
29    0  3.00  4.00  0.904965
30    0  4.00  0.25  0.907915
31    0  4.00  0.50  0.902129
32    0  4

Multi Scale walks

In [9]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Function to load edge list from a file
def load_edgelist(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            edge = line.strip().split()
            edges.append((edge[0], edge[1]))
    return edges

# Function to sample a subgraph using BFS
def sample_subgraph_bfs(graph, start_node=None, number_of_nodes=1000, seed=None):
    if start_node is None:
        nodes = list(graph.nodes())
        random.Random(seed).shuffle(nodes)
        start_node = nodes[0]

    sampled_nodes = set([start_node])
    node_queue = [start_node]

    while len(sampled_nodes) < number_of_nodes and node_queue:
        current = node_queue.pop(0)
        neighbors = sorted(list(graph.neighbors(current)))
        random.Random(seed).shuffle(neighbors)

        for neighbor in neighbors:
            if len(sampled_nodes) >= number_of_nodes:
                break
            if neighbor not in sampled_nodes:
                sampled_nodes.add(neighbor)
                node_queue.append(neighbor)

    return graph.subgraph(sampled_nodes)

# Function to generate edge embeddings
def generate_edge_embeddings(edges, embedding_function):
    edge_embeddings = []
    for u, v in edges:
        u_embedding = embedding_function(u)
        v_embedding = embedding_function(v)
        edge_embedding = np.multiply(u_embedding, v_embedding)
        edge_embeddings.append(edge_embedding)
    return np.array(edge_embeddings)

# Function to split the graph into train and test sets, ensuring connectivity
def split_graph_with_connectivity(graph, fraction_to_remove=0.1, seed=None):
    edge_list = list(graph.edges())
    num_edges_to_remove = int(fraction_to_remove * graph.number_of_edges())

    while True:
        random.Random(seed).shuffle(edge_list)
        edges_to_remove = edge_list[:num_edges_to_remove]
        G_train = graph.copy()
        G_train.remove_edges_from(edges_to_remove)
        if nx.is_connected(G_train):
            break

    G_test = graph.copy()
    G_test.remove_edges_from(G_train.edges())

    return G_train, G_test, edges_to_remove

# Function to generate positive and negative samples
def generate_samples(graph, seed=None):
    positive_samples = list(graph.edges())
    negative_samples = []
    all_nodes = sorted(list(graph.nodes()))

    while len(negative_samples) < len(positive_samples):
        node_pair = random.sample(all_nodes, 2)
        if not graph.has_edge(*node_pair):
            negative_samples.append(tuple(sorted(node_pair)))

    return positive_samples, negative_samples

def generate_node2vec_embeddings_multiscale(graph, dimensions=128, num_walks=10, walk_lengths=[5, 10, 15], p=1.0, q=1.0, seed=None):
    def node2vec_walk(node, length):
        walk = [node]
        for _ in range(length):
            neighbors = list(graph.neighbors(walk[-1]))
            if len(neighbors) > 0:
                if len(walk) == 1:
                    walk.append(random.choice(neighbors))
                else:
                    prev_node = walk[-2]
                    probs = []
                    for neighbor in neighbors:
                        if neighbor == prev_node:
                            probs.append(1 / p)
                        elif graph.has_edge(prev_node, neighbor):
                            probs.append(1)
                        else:
                            probs.append(1 / q)
                    probs = np.array(probs)
                    probs /= np.sum(probs)
                    walk.append(np.random.choice(neighbors, p=probs))
            else:
                break
        return walk

    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.seed(seed)
        random.shuffle(nodes)
        for node in nodes:
            for length in walk_lengths:
                walks.append(node2vec_walk(node, length))

    embeddings = {}
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=1, sg=1, workers=1, seed=seed)
    for node in nodes:
        embeddings[node] = model.wv[node]

    def embedding_function(u):
        return embeddings[u]

    return model, embedding_function


# Grid search over p, q ∈ {0.25, 0.50, 1, 2}
p_values = [0.25, 0.5, 1, 2,3,4]
q_values = [0.25, 0.5, 1, 2,3,4]

# DataFrame to store the results
results_df = pd.DataFrame(columns=['seed', 'p', 'q', 'roc_auc'])

for seed in range(1):
    # Set the new random seed for all functions
    np.random.seed(seed)
    random.seed(seed)

    # Load the data and create the graph
    edge_list = load_edgelist('facebook_combined.txt')# Replace with your actual file path
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # Sample subgraph, split into train/test, generate samples
    subgraph = sample_subgraph_bfs(G, number_of_nodes=300, seed=seed)
    G_train, G_test, _ = split_graph_with_connectivity(subgraph, seed=seed)
    positive_samples_train, negative_samples_train = generate_samples(G_train, seed=seed)
    positive_samples_test, negative_samples_test = generate_samples(G_test, seed=seed)

    # Define the list of walk lengths for multi-scale sampling
    walk_lengths = [5, 10, 15]  # Adjust this list based on the lengths you want to explore

    for p in p_values:
        for q in q_values:
            # Node2Vec embeddings with multi-scale sampling
            _, embedding_function = generate_node2vec_embeddings_multiscale(G_train, p=p, q=q, walk_lengths=walk_lengths, seed=seed)

            # Generate edge embeddings
            edge_embeddings_train = generate_edge_embeddings(positive_samples_train + negative_samples_train, embedding_function)
            edge_embeddings_test = generate_edge_embeddings(positive_samples_test + negative_samples_test, embedding_function)

            # Logistic Regression Model
            labels_train = np.array([1] * len(positive_samples_train) + [0] * len(negative_samples_train))
            lr_clf = LogisticRegressionCV(cv=10, max_iter=2000, scoring="roc_auc", random_state=seed)
            classifier = lr_clf.fit(edge_embeddings_train, labels_train)

            # Evaluate the model
            labels_test = np.array([1] * len(positive_samples_test) + [0] * len(negative_samples_test))
            test_predictions = classifier.predict_proba(edge_embeddings_test)[:, 1]
            test_roc_auc = roc_auc_score(labels_test, test_predictions)

            # Store results
            results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)

# Print DataFrame
print(results_df)

# Find the best combination of p and q based on average ROC AUC
best_combination = results_df.groupby(['p', 'q'])['roc_auc'].mean().idxmax()
print(f"Best combination of p and q: {best_combination}")


C:\Users\ayush\AppData\Local\Temp\ipykernel_45780\3406144043.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'seed': seed, 'p': p, 'q': q, 'roc_auc': test_roc_auc}])], ignore_index=True)


   seed     p     q   roc_auc
0     0  0.25  0.25  0.908735
1     0  0.25  0.50  0.898785
2     0  0.25  1.00  0.890000
3     0  0.25  2.00  0.898589
4     0  0.25  3.00  0.877526
5     0  0.25  4.00  0.874199
6     0  0.50  0.25  0.911587
7     0  0.50  0.50  0.914341
8     0  0.50  1.00  0.896458
9     0  0.50  2.00  0.904113
10    0  0.50  3.00  0.893032
11    0  0.50  4.00  0.904506
12    0  1.00  0.25  0.910800
13    0  1.00  0.50  0.906735
14    0  1.00  1.00  0.915062
15    0  1.00  2.00  0.906653
16    0  1.00  3.00  0.895638
17    0  1.00  4.00  0.880477
18    0  2.00  0.25  0.910964
19    0  2.00  0.50  0.911472
20    0  2.00  1.00  0.919864
21    0  2.00  2.00  0.911423
22    0  2.00  3.00  0.914636
23    0  2.00  4.00  0.891098
24    0  3.00  0.25  0.902867
25    0  3.00  0.50  0.910948
26    0  3.00  1.00  0.906047
27    0  3.00  2.00  0.911685
28    0  3.00  3.00  0.907637
29    0  3.00  4.00  0.904539
30    0  4.00  0.25  0.914963
31    0  4.00  0.50  0.921208
32    0  4